# Transfer Learning Image Classifier
## What is this notebook?
In this notebook we will train a neural network to classify the images in our synthetic dataset. It should be able to load an image and determine whether it contains an 'A', 'B', or 'C'.

Note: This is different from an object detector, which can find multiple objects in a single image and put them in bounding boxes. A classifier only gives one label for the entire image.

We could create a simple convolutional neural network from scratch, but in practice, it is much more common to start with a pre-trained neural network and re-train it to work with a new dataset. In order to do this re-training, called "transfer learning", we will start with a model on [TensorFlow Hub](https://www.tensorflow.org/hub) that has been trained on the ImageNet dataset. We will then add a couple layers to the end of the network and train it.

## Inspiration
This notebook and its code were inspired by the [Transfer learning with TensorFlow Hub](https://www.tensorflow.org/tutorials/images/transfer_learning_with_hub) tutorial. You can refer to that if you would like to learn more about it.



## Setup & Imports
First we'll make sure TensorFlow and TensorFlow Hub are set up and that we import all of the modules we need.
### Action Required
- If you have not already installed tensorflow, tensorflow-hub, and matplotlib in your environment, do so here by uncommenting the appropriate lines
- If you have a CUDA GPU and wan to use Tensorflow GPU, use the "pip install tensorflow-gpu" line, otherwise use "pip install tensorflow"

In [ ]:
# Install tensorflow module (use the second line for Tensorflow GPU, if you have CUDA)
#!pip install "tensorflow>=2.0.0"
#!pip install "tensorflow-gpu>=2.0.0"

# Install tensorflow_hub module
#!pip install "tensorflow_hub>=0.6.0"

# Install matplotlib
#!pip install "matplotlib"

In [ ]:
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Load Training, Validation, and Test Datasets
We want to load our training, validation, and test images into ImageDataGenerators, which will be used to train/validate our TensorFlow model.

### Action Required
- Update the **train_dir**, **validation_dir**, & **test_dir** directory paths below to match where you've saved your datasets

In [ ]:
# Define the IMAGE_SHAPE, train_dir, validation_dir, and test_dir
IMAGE_SHAPE = (224, 224)
data_path = 
train_dir      = data_path + 'train'
validation_dir = data_path + 'val'
test_dir       = data_path + 'test'

In [ ]:
# Create a training image generator and data generator
train_image_generator = ImageDataGenerator(rescale=1./255)
train_data_gen = train_image_generator.flow_from_directory(directory=train_dir, shuffle=True, target_size=IMAGE_SHAPE, class_mode='categorical')

In [ ]:
# Create a validation image generator and data generator
validation_image_generator = ImageDataGenerator(rescale=1./255)
validation_data_gen = validation_image_generator.flow_from_directory(directory=validation_dir, shuffle=True, target_size=IMAGE_SHAPE, class_mode='categorical')

In [ ]:
# Create a test image generator and data generator
test_image_generator = ImageDataGenerator(rescale=1./255)
test_data_gen = test_image_generator.flow_from_directory(directory=test_dir, shuffle=True, target_size=IMAGE_SHAPE, class_mode='categorical')

## Display Sample Images
Now we'll display a few sample images from the Training and Validation datasets.

In [ ]:
def display_samples(data_gen, title):
    # Make a lookup dictionary for class indexes
    classes = dict()
    for key, val in data_gen.class_indices.items():
        classes[val] = key

    # Get sample images and labels
    sample_images, sample_labels = next(data_gen)

    # Plot the images and labels
    plt.figure(figsize=(10,3))
    plt.subplots_adjust(hspace=0.5)
    for n in range(5):
        plt.subplot(1,5,n+1)
        plt.imshow(sample_images[n])
        plt.title(classes[np.argmax(sample_labels[n])])
        plt.axis('off')
    _ = plt.suptitle(title)

display_samples(train_data_gen, 'Sample Training Images')
display_samples(validation_data_gen, 'Sample Validation Images')
display_samples(test_data_gen, 'Sample Test Images')

## Creating the TensorFlow Model
Now we'll create a new TensorFlow model based on the "imagenet/mobilenet_v2_100_224/feature_vector". This is a pre-trained model, but it has never been trained on 3D letters before, so we'll be retraining it for our purposes further below.

Description: "Feature vectors of images with MobileNet V2 (depth multiplier 1.00) trained on ImageNet (ILSVRC-2012-CLS)."

Source: https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4

In [ ]:
# Create a TensorFlow model for classifying images
num_classes = len(train_data_gen.class_indices)
model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4",
                   trainable=False, input_shape=IMAGE_SHAPE+(3,)),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Print out the model summary
model.summary()

In [ ]:
# Compile the model
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

## Training
In the block below, we'll train our neural network. In my experiments, I got .93 to .95 accuracy on the validation set.

In [ ]:
# Calculate the train & val steps per epoch
train_steps_per_epoch = np.ceil(train_data_gen.samples/train_data_gen.batch_size)
val_steps_per_epoch = np.ceil(validation_data_gen.samples/validation_data_gen.batch_size)

# Train the model
epochs = 4
history = model.fit(train_data_gen, epochs=epochs,
                    steps_per_epoch=train_steps_per_epoch,
                    validation_data=validation_data_gen,
                    validation_steps=val_steps_per_epoch)

## Inference
Now that our model is trained, let's run some of our test images through it.

In [ ]:
# Make a lookup numpy array for class names
class_names = np.empty([len(test_data_gen.class_indices)], dtype=str, order='C')
for key, val in test_data_gen.class_indices.items():
    class_names[val] = key

# Get a test batch of images and labels
test_data_gen.reset()
image_batch, label_batch = next(test_data_gen)

# Predict the class of each image using our model
predicted_batch = model.predict(image_batch)
predicted_id = np.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]
label_id = np.argmax(label_batch, axis=-1)

## Show Predictions
Now we'll display predictions for our first 30 test images. If training worked properly, most of them should be correct, and the ones that fail should be fairly challenging!

In [ ]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(min(30, len(image_batch))):
    plt.subplot(6,5,n+1)
    plt.imshow(image_batch[n])
    color = "green" if predicted_id[n] == label_id[n] else "red"
    plt.title(predicted_label_batch[n].title(), color=color)
    plt.axis('off')
_ = plt.suptitle("Model predictions (green: correct, red: incorrect)")